In [1]:
from mypackage import scraping_functions as sf
from mypackage import predict_functions as pf
from mypackage import cleaning_functions as cf
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup, Comment
import re
import time
from math import inf

In [2]:
game_url = 'https://www.pro-football-reference.com/boxscores/202309070kan.htm'
test_data = sf.scrape_game_data(game_url)
train_data = pd.read_csv('../nfl_scraping_project/mypackage/data/week_10_2023_plays.csv')

Chiefs vs. Lions 
 total plays: 132


In [3]:
accuracy, post_data = pf.predict_wins(test_data, train_data)

accuracy: 0.8702290076335878


In [ ]:
# Prepare Training Data\
train_data = train_data.dropna(subset=['Down'])
train_data = train_data[train_data['Down'] != '']
train_data['seconds_left'] = train_data.apply(cf.seconds_left, axis=1)
train_data['score_diff'] = train_data.iloc[:,5].astype(float) - train_data.iloc[:,6].astype(float)
train_data['adjusted_score'] = train_data.apply(cf.adjusted_score_calc, axis=1)
train_data['win_team'] = train_data.apply(lambda row: cf.win(pd.DataFrame(row).transpose()), axis=1)

y_train = train_data['win_team']
y_train = y_train.apply(lambda x:float(x[0]))

columns_to_drop = [5, 6]
additional_columns_to_drop = ['Quarter', 'Detail', 'Location', 'Time', 'win_team', 'play_start_time', 'game_id', 'win_team', 'posession']

train_data = train_data.drop(columns=train_data.columns[columns_to_drop])
train_data = train_data.drop(columns=additional_columns_to_drop)
train_data['field_side'] = pd.Categorical(train_data['field_side'])
train_data['Play_Type'] = pd.Categorical(train_data['Play_Type'])
train_data['possession'] = pd.Categorical(train_data['possession'])
train_data['Down'] = pd.Categorical(train_data['Down'])
train_data['ToGo'] = pd.to_numeric(train_data['ToGo'], errors='coerce')
train_data['ToGo'] = train_data['ToGo'].astype(float)
train_data['EPB'] = train_data['EPB'].astype(float)
train_data['EPA'] = train_data['EPA'].astype(float)
train_data = pd.get_dummies(train_data, columns=['Play_Type', 'Down', 'field_side', 'possession'])
train_data = train_data.fillna(0)

In [ ]:
test_data = test_data.dropna(subset=['Down'])
test_data = test_data[test_data['Down'] != '']
test_data['seconds_left'] = test_data.apply(cf.seconds_left, axis=1)
test_data['score_diff'] = test_data.iloc[:,5].astype(float) - test_data.iloc[:,6].astype(float)
test_data['adjusted_score'] = test_data.apply(cf.adjusted_score_calc, axis=1)
test_data['win_team'] = test_data.apply(lambda row: cf.win(pd.DataFrame(row).transpose()), axis=1)
test_data['win_team'] = test_data.apply(lambda row: cf.win(pd.DataFrame(row).transpose()), axis=1)

y = test_data['win_team']
columns_to_drop = [5, 6]
additional_columns_to_drop = ['Quarter', 'Detail', 'Location', 'Time', 'win_team', 'play_start_time', 'game_id', 'win_team']
new_data = test_data.drop(columns=test_data.columns[columns_to_drop])
# Optionally, you can also drop additional columns
additional_columns_to_drop = ['Quarter', 'Detail', 'Location', 'Time', 'win_team', 'play_start_time']
new_data = new_data.drop(columns=additional_columns_to_drop)

new_data['field_side'] = pd.Categorical(new_data['field_side'])
new_data['Play_Type'] = pd.Categorical(new_data['Play_Type'])
new_data['possession'] = pd.Categorical(new_data['possession'])
new_data['Down'] = new_data['Down'].astype(float)
new_data['Down'] = pd.Categorical(new_data['Down'])
new_data['ToGo'] = pd.to_numeric(new_data['ToGo'], errors='coerce')
new_data['ToGo'] = new_data['ToGo'].astype(float)
new_data['EPB'] = new_data['EPB'].astype(float)
new_data['EPA'] = new_data['EPA'].astype(float)
new_data['yardline'] = new_data['yardline'].astype(float)
new_data = pd.get_dummies(new_data, columns=['Play_Type', 'Down', 'field_side', 'possession'])

y = y.apply(lambda x:float( x[0]))
new_data = new_data.fillna(0)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=400, random_state=42)
rf_classifier.fit(train_data, y_train)
y_pred = rf_classifier.predict(new_data)

accuracy = accuracy_score(y, y_pred)
print(f'accuracy: {accuracy}')

In [ ]:
# game_links = [
#     'https://www.pro-football-reference.com/boxscores/202309110nyj.htm',
#     'https://www.pro-football-reference.com/boxscores/202309070kan.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100atl.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100chi.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100cle.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100clt.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100den.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100min.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100nor.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100nwe.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100nyg.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100pit.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100rav.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100sdg.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100sea.htm',
#     'https://www.pro-football-reference.com/boxscores/202309100was.htm'    
#     ]

# game_file_path ='../nfl_scraping_project/mypackage/data/week_10_2023_games.csv'
# data_file_path = '../nfl_scraping_project/mypackage/data/week_10_2023_plays.csv'

# sf.scrape_games(game_links, data_file_path, game_file_path)

# # games = {
# # 'game_id': [], 
# # # 'team1': [], 
# # # 'team2': [],
# # 'link': []
# # }

# # data = pd.DataFrame({
# #         'Quarter': [], 
# #         'Time': [], 
# #         'Down': [], 
# #         'ToGo': [], 
# #         'Location': [], 
# #         'away': [], 
# #         'home': [], 
# #         'Detail': [], 
# #         'EPB': [],
# #         'EPA': [], 
# #         'field_side': [], 
# #         'yardline': [], 
# #         'play_start_time': [], 
# #         'Play_Type': [], 
# #         'posession': [], 
# #         'Yardage': []
# #                     })
# # id = 1
# # for link in game_links:
# #     game_data = sf.scrape_game_data(link)
# #     game_data['game_id'] = id
# #     team1 = game_data.columns[5]
# #     team2 = game_data.columns[6]
# #     # game_data = game_data.rename(columns={game_data.columns[5]: 'team1', 
# #     #                             game_data.columns[6]: 'team2'})
# #     games['game_id'].append(id)
# #     # games['team1'].append(team1)
# #     # games['team2'].append(team2)
# #     games['link'].append(link)
# #     data = pd.concat([data, game_data], axis=0)
# #     id = id + 1
# #     time.sleep(10)
# # games_df = pd.DataFrame(games)
# # # data.to_csv(data_file_path,index=False)
# # # games_df.to_csv(game_file_path, index=False)
# # print('done')

In [ ]:
team_keys = {
    'DET': 'Lions',
    'KAN': 'Chiefs',
    'ATL': 'Falcons',
    'CAR': 'Panthers',
    'CLE': 'Browns', 
    'CIN': 'Bengals', 
    'IND': 'Colts', 
    'JAX': 'Jaguars',
    'MIN': 'Vikings', 
    'TAM': 'Buccaneers',
    'NOR': 'Saints', 
    'TEN': 'Titans',
    'PIT': 'Steelers', 
    'SFO': '49ers', 
    'BAL': 'Ravens',
    'HOU': 'Texans', 
    'WAS': 'Commanders',
    'ARI': 'Cardinals',
    'CHI': 'Bears', 
    'GNB': 'Packers',
    'DEN': 'Broncos', 
    'LVR': 'Raiders', 
    'NWE': 'Patriots', 
    'PHI': 'Eagles', 
    'LAC': 'Chargers', 
    'MIA': 'Dolphins',
    'SEA': 'Seahawks',
    'LAR': 'Rams',
    'NYG': 'Giants',
    'DAL': 'Cowboys',
    'NYJ': 'Jets', 
    'BUF': 'Bills'
}
pbp_data = []
game_url = 'https://www.pro-football-reference.com/boxscores/202309110nyj.htm'
r = requests.get(game_url)
game_page_soup = BeautifulSoup(r.text, 'html.parser')
# scraping drive data for home and away team
home_drives = sf.get_drive_table('home', game_page_soup)
vis_drives = sf.get_drive_table('vis', game_page_soup)
# getting home and away team variables
home_team = home_drives['team'][0]
vis_team = vis_drives['team'][0]
home_vis = {home_team: 'home', vis_team: 'away'}

teams = [home_team, vis_team]
drives = pd.concat([home_drives, vis_drives], axis=0)
drives['Quarter'] = drives['Quarter'].astype(int)
drives['minute'] = drives['Time'].str.extract(r'([0-9]+):').astype(int)
drives['seconds'] = drives['Time'].str.extract(r'[0-9]+:([0-9]+)').astype(int)
drives['seconds_ratio'] = (drives['seconds'] / 60).astype(float)
drives['Numeric_time'] = drives['minute'] + drives['seconds_ratio']
drives = drives.sort_values(by=['Quarter', 'Numeric_time'], ascending=[True, False]).reset_index()
drives['drive_time'] = cf.elapsed_time(drives['Numeric_time'])
drives['drive_start_time'] = cf.game_time(drives['drive_time'])
drives = drives.drop(columns=['minute', 'seconds', 'seconds_ratio', 'index', '#', 'Numeric_time', 'drive_time'])
# scraping pbp data
pbp_data = sf.scrape_pbp(game_page_soup)
# Setting up receiving and kicking teams
coin_toss = pbp_data.iloc[0]
pbp_data = pbp_data.drop(0)
coin_toss = coin_toss[7]
teams = re.findall(r'\b[A-Z][a-zA-Z]*\b', coin_toss)
match = re.search(r"(\w+)\s+to\s+receive\s+the\s+opening\s+kickoff", coin_toss)
if match:
    receiving_team = match.group(1)
# dropping timeouts
pbp_data = pbp_data[pbp_data['Location'].str.strip() != '']
pbp_data = pbp_data.dropna(subset=['Location'])
print(f'{home_team} vs. {vis_team} \n total plays: {len(pbp_data)}')
# General Cleaning
pbp_data['Quarter'] = pbp_data['Quarter']
pbp_data['Quarter'] = [x if x != 'OT' else int(5) for x in pbp_data['Quarter']]
pbp_data['field_side'] = pbp_data['Location'].str.extract(r'([A-Z]+)')
pbp_data['field_side'] = [home_vis[team_keys[x]] if x == teams[0] else home_vis[team_keys[x]] for x in pbp_data['field_side']]
pbp_data['yardline'] = pbp_data['Location'].str.extract(r'([0-9]+)')
pbp_data['yardline'] = pbp_data['yardline'].astype(int)
pbp_data['minute'] = pbp_data['Time'].str.extract(r'([0-9]+):').astype(int)
pbp_data['seconds'] = pbp_data['Time'].str.extract(r'[0-9]+:([0-9]+)').astype(int)
pbp_data['seconds_ratio'] = (pbp_data['seconds'] / 60).astype(float)
pbp_data['Numeric_time'] = pbp_data['minute'] + pbp_data['seconds_ratio']
pbp_data['play_time'] = cf.elapsed_time(pbp_data['Numeric_time'])
pbp_data['play_start_time'] = cf.game_time(pbp_data['play_time'])
pbp_data = pbp_data.drop(columns=['minute', 'seconds', 'seconds_ratio','Numeric_time', 'play_time'])
pbp_data['Play_Type'] = pbp_data['Detail'].apply(cf.play_type)
